In  this  project,  I am  building  a  simple  Information  Retrieval  system  based  on  Boolean  AND operation.   The  main  idea  is  to  create  a  fast  retrieval  system  using  inverse  indexing  and create relevant posting list.

The first part of the code is to load the dataset. If we are using google colab, we authorise the access by google account and change to the google drive where the dataset resides. I am making use of the nltk package for stemmer and lemmatization of tokens.

In [1]:
import sys, os
import nltk

if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/gdrive')
    os.chdir('/content/gdrive')

#Download stemmer (punkt) and lemmatiser (wordnet)
nltk.download('punkt')
nltk.download('wordnet')

#Using SnowballStemmer: Reason -> https://towardsdatascience.com/stemming-lemmatization-what-ba782b7c0bd8
from nltk.stem.snowball import SnowballStemmer
stemmer=SnowballStemmer('english')

#Using WordNet Lemmatiser
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

#Load stopwords
## Corrected stopword file name
for p,d,f in os.walk('.'):
  if 'stopwords.txt' in f:
    print(f)
    fullpath=os.path.join(p,'stopwords.txt')
    break
fstop=open(fullpath,encoding='utf-8-sig')
stopw=fstop.read().split()



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
['Proj2.ipynb', 'stopwords.txt', 'Proj3-benchmark.ipynb']


In [2]:
#Stemmer test
a=stemmer.stem('transferring')
print(a)
#Lemma test
a=lemmatizer.lemmatize('feet')
print(a)

transfer
foot


The following function searches the specified dataset folder name and returns the full directory path of the dataset, "fullpath" with a list of filename in a list, "docfilename".

In [3]:
def directory_list(datasetname):
  fullpath=''
  for p,d,f in os.walk('.'):
    if datasetname in d:
      fullpath=os.path.join(p,datasetname)
      break
  os.chdir(fullpath)
  docfilename=os.listdir()
  return fullpath,docfilename

The following searches the dataset named "HillaryEmails" and return full path "path" to the dataset and a list of documents text "docfilename" in the directory.

In [4]:
path,docfilename=directory_list('HillaryEmails')
print("The path name is: ", path)
print("The list of documents in the dataset: ", docfilename)

The path name is:  ./MyDrive/Colab Notebooks/HillaryEmails
The list of documents in the dataset:  ['1053.gdoc', '5547.gdoc', '7422.gdoc', '3644.gdoc', '7436.gdoc', '4881.gdoc', '1735.gdoc', '5235.gdoc', '4895.gdoc', '7344.gdoc', '3122.gdoc', '5553.gdoc', '4659.gdoc', '7350.gdoc', '1721.gdoc', '2228.gdoc', '6728.gdoc', '1047.gdoc', '3650.gdoc', '3136.gdoc']


In [5]:
docidlen=len(docfilename)
print("The number of documents in the dataset: ", docidlen)
for i,d in enumerate(docfilename):
  print(i, d)

The number of documents in the dataset:  20
0 1053.gdoc
1 5547.gdoc
2 7422.gdoc
3 3644.gdoc
4 7436.gdoc
5 4881.gdoc
6 1735.gdoc
7 5235.gdoc
8 4895.gdoc
9 7344.gdoc
10 3122.gdoc
11 5553.gdoc
12 4659.gdoc
13 7350.gdoc
14 1721.gdoc
15 2228.gdoc
16 6728.gdoc
17 1047.gdoc
18 3650.gdoc
19 3136.gdoc


The following function "processString" processes an input string by removing punctuations, lowering the case, removing stop words, lemmatising and stemming.

In [6]:
  def processString(String, stopw):

      global orisz
      orisz+=sys.getsizeof(String)

      #Removes symbols completely
      for i in ',!@#$%^&*()-=+_":\';<>/\\.?':
          String=String.replace(i,'')

      if bench:
          global rmpuncsz
          rmpuncsz+=sys.getsizeof(String)

      #Convert all to lowercase
      String=String.lower()

      #Remove stop words (https://code.google.com/archive/p/stop-words/)
      ##Split words into a list with separate entries (separated by space)
      String=String.split()
      for s in stopw:
          for i in range(len(String)-1,-1,-1):
              if String[i]==s:
                  String.pop(i)

      if bench:
          global stopsz
          stopsz+=sys.getsizeof(' '.join(String))

          global stemdur
          #Using snowball stemmer
          temp=time.time()
          S0=[stemmer.stem(i) for i in String]
          stemdur[0]+=time.time()-temp

          #Porter Stemmer
          temp=time.time()
          S1=[porter.stem(i) for i in String]
          stemdur[1]+=time.time()-temp

          #Lancaster Stemmer
          temp=time.time()
          S2=[lancaster.stem(i) for i in String]
          stemdur[2]+=time.time()-temp

          global stemsz
          str1=' '.join(S0)
          str2=' '.join(S1)
          str3=' '.join(S2)
          stemsz[0]+=sys.getsizeof(str1)
          stemsz[1]+=sys.getsizeof(str2)
          stemsz[2]+=sys.getsizeof(str3)
          
          global lemmadur

          #Using wordnet lemmatiser
          temp=time.time()
          S00=[lemmatizer.lemmatize(i) for i in S0]
          lemmadur[0,0]+=time.time()-temp

          temp=time.time()
          S10=[lemmatizer.lemmatize(i) for i in S1]
          lemmadur[1,0]+=time.time()-temp

          temp=time.time()
          S20=[lemmatizer.lemmatize(i) for i in S2]
          lemmadur[2,0]+=time.time()-temp
          
          #Using SpaCy lemmatizer
          temp=time.time()
          S01=' '.join([token.lemma_ for token in spacymod(str1)]) 
          lemmadur[0,1]+=time.time()-temp

          temp=time.time()
          S11=' '.join([token.lemma_ for token in spacymod(str2)])
          lemmadur[1,1]+=time.time()-temp
          
          temp=time.time()
          S21=' '.join([token.lemma_ for token in spacymod(str3)])
          lemmadur[2,1]+=time.time()-temp

          #Using TextBlob lemmatizer
          temp=time.time()
          S02=" ".join([w.lemmatize() for w in TextBlob(str1).words])
          lemmadur[0,2]+=time.time()-temp

          temp=time.time()
          S12=" ".join([w.lemmatize() for w in TextBlob(str2).words])
          lemmadur[1,2]+=time.time()-temp

          temp=time.time()
          S22=" ".join([w.lemmatize() for w in TextBlob(str3).words])
          lemmadur[2,2]+=time.time()-temp

          String=' '.join(S00)
          S1=' '.join(S10)
          S2=' '.join(S20)
          global lemmasz

          lemmasz[0,0]+=sys.getsizeof(String)
          lemmasz[1,0]+=sys.getsizeof(S1)
          lemmasz[2,0]+=sys.getsizeof(S2)
          lemmasz[0,1]+=sys.getsizeof(S01)
          lemmasz[1,1]+=sys.getsizeof(S11)
          lemmasz[2,1]+=sys.getsizeof(S21)
          lemmasz[0,2]+=sys.getsizeof(S02)
          lemmasz[1,2]+=sys.getsizeof(S12)
          lemmasz[2,2]+=sys.getsizeof(S22)
      else:
          #Using snowball stemmer
          S0=[stemmer.stem(i) for i in String]

          String=' '.join(S0)

      return String

A benchmark has also been written to show the difference in sizes and time taken when different stemmers and lemmatisers are used on the dataset.

In [11]:
# Benchmark Option Toggle
bench=1

## Benchmarking varible initialisation
if bench:
    import numpy as np
    import time
    stemsz=np.zeros(3)
    stopsz=0
    lemmasz=np.zeros((3,3))
    orisz=0
    stemdur=np.zeros(3,dtype='float64')
    lemmadur=np.zeros((3,3),dtype='float64')
    rmpuncsz=0

    #Imports Porter Stemmer and Lancaster Stemmer for benchmarking
    from nltk.stem import PorterStemmer, LancasterStemmer
    porter=PorterStemmer()
    lancaster=LancasterStemmer()

    #Important to do this or run spacy.cli.download('en') from command line manually
    '''This code will have error if is not running with appropriate permissions -> Will need to test directly with colab
        !python -m spacy download en
        OR'''
    #Imports spacy lemmatiser
    import spacy
    spacy.cli.download('en')
    spacymod=spacy.load('en')
    
    #Imports Textblob lemmatiser
    from textblob import TextBlob

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [21]:
if bench:
    for f in range(len(docfilename)):
        print("[%3.1f%%] Now processing: %s"%((f+1.0)/len(docfilename)*100,docfilename[f]))
        processString(open(os.path.join('.',docfilename[f]),'r',encoding='utf-8-sig').read(),stopw)
    print()
    print(orisz)
    print(rmpuncsz)
    print(stopsz)
    print(stemsz)
    print(stemdur)
    print(lemmasz)
    print(lemmadur)

[33.3%] Now processing: 7945.txt
[66.7%] Now processing: 5642.txt
[100.0%] Now processing: 7942.txt

543048
526584
353720
[307096. 307072. 263848.]
[0.38859606 0.57730627 0.41971993]
[[306752. 306960. 306576.]
 [306744. 307000. 306568.]
 [263240. 264720. 263048.]]
[[0.10349345 3.39737725 0.31074262]
 [0.09578085 3.26415825 0.29463887]
 [0.10128593 3.29457283 0.27698231]]


Benchmark turned off as it is not needed after this

In [22]:
bench=0

In [23]:
# processString testing
processString("tokenizing is a way to do shorten and small pieces of words", stopw)

'token shorten small piec'

The following function "tokenizer" reads in the path and the docfilename list which contains the list of name of the text documents to process. It uses the processString function to process the string by removing the stop words, symbols, stemmer and lemmatization. The function returns all the tokens in a document.

In [25]:
def tokenizer(path,docfilename):
  t=dict()
  tchar=['','\n','\t','\n\t','\t\n']
  os.chdir(path)
  for index, d in enumerate(docfilename):
    fpath=path+"/"+d
    f=open(d)
    a=f.read()
    a=processString(a, stopw)
    a=a.split('\n')
    a=' '.join(a).split(' ')
    for i in a:
      if i not in tchar:
        if i not in t:
          t[i]=list()
        if index not in t[i]:
          t[i].append(index)
  return t


The following "sort_token" function sorts the token in alphabetical order and subsequently sorts the values which are the document id or indices in the key which is the token. 

The mergeposting function merges two lists together by using pointers to each list and compares the values of the index in the two lists. If the two lists are equal, the value will be added to the postlist. If a list value is smaller than the other, its index will be increased. 

The "intersect" function will extract two lists from the dictionary with reference to two words. It produces the merge posting list for the two words.  

In [26]:
def sort_token(token):
  sdict=dict()
  skey=sorted(token)
  for s in skey:
    sdict[s]=sorted(token[s])
  return sdict

def mergeposting(list1,list2):
  postlist=list()
  i=0
  j=0
  len1=len(list1)
  len2=len(list2)
    
  while i<len1 and j<len2:
    #print(i," ",j)
    if list1[i]==list2[j]:
      postlist.append(list1[i])
      i+=1
      j+=1
    else:
      if list1[i]<list2[j]:
        i+=1
      else:
        j+=1
  return postlist

def intersect(sdict,word1,word2):
  postlist=list()
  try:
    list1=sdict[word1]
    list2=sdict[word2]
    postlist=mergeposting(list1,list2)
  except:
    print("words not valid")
  return postlist



The following section takes into consideration the organisation of dictionary as a string. 

The code cell immediately below is used for binary search of terms from the dictionary which is stored as a string.

In [27]:
## Binary search algorithm
# string : the dictionary stored as string
# arr : pointers to beginning of words in the string
# l : lower bound (for search) initially defined as the beginning of string
# r : upper bound (for search) initially defined as the end of string
# word : the word that is to be searched

def searchForTerm ( string, arr, l, r, word): 
    # Check base case 
    if r >= l: 
        mid = l + (r - l) // 2
  
        # If element is present at the middle itself 
        if string[arr[mid]:arr[mid+1]] == word: 
            return mid 
          
        # If element is smaller than mid, then it  
        # can only be present in left subarray 
        elif string[arr[mid]:arr[mid+1]] > word: 
            return searchForTerm(string, arr, l, mid-1, word) 
  
        # Else the element can only be present  
        # in right subarray 
        else: 
            return searchForTerm(string, arr, mid + 1, r, word) 
  
    else: 
        # Element is not present in the array 
        return -1

The following cell is a new intersect function which finds the posting list for each of the words in the query and then merges them. Here the dictionary is a string and terms are searched through binary search as mentioned above

In [37]:
## Function to find the posting list that is common to both the words being searched for
# positionDict : Its a dictionary that contains the pointers to the words as keys
# and postings as values
# joinedKeys : dictionary stored as string
# word1 : first word to be searched
# word2 : second word to be searched

def intersectNew(positionDict,joinedKeys,word1,word2):
  postlist=list()
  #getting the pointer to each term in the dictionary string
  keyValues = [*positionDict]
  l = 0
  r = len(keyValues)-1
  index1 = searchForTerm(joinedKeys, keyValues, l, r, word1)
  index2 = searchForTerm(joinedKeys, keyValues, l, r, word2)
  if (index1 and index2 > 0):
    list1 = positionDict[keyValues[index1]]
    list2 = positionDict[keyValues[index2]]
  try:
    postlist=mergeposting(list1,list2)
  except:
    print("words not valid")
  return postlist


The following 2 functions convert the dictionary into string and create a new dictionary item with (key,value) as (pointer,postings) for every word in the dictionary

In [29]:
## converts dictionary into string by taking the key values of sorted token pairs
# and joining them as string
# sdict: sorted token pair
def getJoinedStringOfDict(sdict):
  return ''.join([*sdict])

In [30]:
## Finds the position of the word in the string using the counter, which gets updated
# after every word is processed by adding the length of the word to it and the postings
# are copied to the values of the pointers (keys). 
# The final format is : positionDict ==> (key, value):(pointer, posting list)

def getPositionDict(sdict):
  count = 0
  positionDict = dict()
  for i in [*sdict]:
    positionDict[count] = sdict[i]
    count = count+len(i)
  
  return positionDict

The following cells are codes for blocked storage by grouping the terms in the string into of size $k$ where $k$ = 4

In [32]:
## Binary search algorithm to find the block to which a word belongs
# string : the dictionary stored as string with length of words in block manner
# arr : pointers to beginning of blocks in the string
# l : lower bound (for search) initially defined as the beginning of string
# r : upper bound (for search) initially defined as the end of string
# word : the word that is to be searched

def getBlockID (string, arr, l, r, word): 
    # Check base case 
    if r >= l: 
        mid = l + (r - l) // 2
        start = arr[mid]+2
        end = start+(int(string[arr[mid]:arr[mid]+2])) 
        firstWord = string[start:end]

        startNext = arr[mid+1]+2
        endNext = startNext + (int(string[arr[mid+1]:arr[mid+1]+2]))
        firstWordNext = string[startNext:endNext]
        
        #pdb.set_trace()
        # If element is present at the middle itself 
        if (firstWord == word) or (word>firstWord and word<firstWordNext):
            return mid 
          
        # If element is smaller than mid, then it  
        # can only be present in left subarray 
        elif firstWord > word:
            return getBlockID(string, arr, l, mid-1, word) 
  
        # Else the element can only be present  
        # in right subarray 
        elif (word>firstWord and word>firstWordNext): 
            return getBlockID(string, arr, mid + 1, r, word) 
  
    else: 
        # Element is not present in the array 
        return -1

In [97]:
## After it is determined which block to get the information from
# we find the position of the word in the block that matches the 
# query word
# blockString: block string containing length of the words and words
# wordToTest : query word
# returns the position of the word

def getPositionOfWord (blockString, wordToTest):
    end = 0
    for i in range(4):
        start = end+2
        end = start+(int(blockString[end:end+2]))
        #pdb.set_trace()
        word = blockString[start:end]
        if (word == wordToTest):
            return i 
    return -1

In [105]:
## This function is to find the postings list of the word that is in the query
# from the dictionary stored in blocked storage fashion
# positionDictNew : Its a dictionary that contains the pointers to the blocks as keys
# and postings of the block as values
# joinedKeysNew : dictionary stored as string with their length
# word: word to be searched
import pdb
def getPostList (positionDictNew, joinedKeysNew, word):
    pointers = [*positionDictNew]
    left = 0
    right = len(pointers)-1
    blockID = getBlockID(joinedKeysNew, pointers, left, right, word)
    if blockID > 0:
        blockString = joinedKeysNew[pointers[blockID]:pointers[blockID+1]]
        positionOfWord = getPositionOfWord (blockString, word)
        #print(blockString)
        #print(positionOfWord)
        #pdb.set_trace()

        if positionOfWord > 0 :
            postList = positionDictNew[pointers[blockID]][positionOfWord]
            return postList
        else:
            return -1
    else:
        return -1

In [114]:
## Function to find the posting list that is common to both the words being searched for
# This is for the blocked storage type
# positionDictNew : Its a dictionary that contains the pointers to the blocks as keys
# and postings of the block as values
# joinedKeysNew : dictionary stored as string with their length
# word1 : first word to be searched
# word2 : second word to be searched

def intersectNewLength(positionDictNew, joinedKeysNew, word1, word2):
    postlist = list()
    list1 = getPostList(positionDictNew, joinedKeysNew, word1)
    list2 = getPostList(positionDictNew, joinedKeysNew, word2)
    
    try:
        postlist=mergeposting(list1,list2)
    except:
        print("words not valid")
    return postlist

In [39]:
## converts dictionary into string along with length of each word by taking the 
# key values of sorted token pairs and joining them as string with length 
# sdict: sorted token pair
# returns string. eg. stringDict = ['hi', 'bye', 'take', 'care']
# output --> 02hi03bye04take04care

def getJoinedLengthStringOfDict(sdict):
  stringDict = [*sdict]
  listOfStrings = list()
  for i in range(len(stringDict)):
    if (len(stringDict[i])>9):
      lengthOfString = str(len(stringDict[i]))
    else:
      lengthOfString = '0'+str(len(stringDict[i]))
    listOfStrings.append(lengthOfString+stringDict[i])
  return ''.join(listOfStrings)

In [40]:
## Finds the position of the block (of size 4) in the string using the counter, which gets updated
# after every word is processed by adding the length of the block to it and the postings
# are copied (for the whole block which are later retrieved for specific word while testing) 
# to the values of the pointers (keys). 
# The final format is : positionDict ==> (key, value):(pointer to block, posting list of block)

def getPositionDictNew(sdict):
  count = 0
  k = 4   #4 block encoding 
  positionDictNew = dict()
  sdictKeys = [*sdict]
  for i in range(len(sdictKeys)):
    if i%k == 0:
      try:
        positionDictNew[count] = [sdict[sdictKeys[i+d]] for d in range(k)]
      except:
        positionDictNew[count] = [sdict[sdictKeys[i+d]] for d in range(len(sdictKeys)%k)]
    count = count+(len(sdictKeys[i]))+2 
  return positionDictNew

The retrieval function takes in the string of query, process the query by processString function and split it into words. It pair the words and uses the merge listing to produce merge posting list.

In [115]:
def retrieval(query,sdict,optimise):
    query=processString(query,stopw)
    splitquery=query.split(' ')
    
    if optimise == 1:
        #Get dictionary as string
        joinedString = getJoinedStringOfDict(sdict)
        #Get dictionary of pointers and postings as (key,value)
        positionDict = getPositionDict(sdict)

    elif optimise == 2:  
        #Get dictionary as string with length of word before each word
        joinedStringLength = getJoinedLengthStringOfDict(sdict)
        #Get dictionary of pointers to blocks and postings of block as (key, values)
        positionDictNew = getPositionDictNew(sdict)
        
    
    lensplit=len(splitquery)
    
    if lensplit>1:
        mergepost=None
        for i in range(lensplit-1):
            if optimise == 1:
                #Get from the intersect function with dictionary as string storage
                postlist = intersectNew(positionDict, joinedString, splitquery[i], splitquery[i+1])
            elif optimise == 2:
                #Get from the intersect function with blocked storage
                postlist = intersectNewLength(positionDictNew, joinedStringLength, splitquery[i], splitquery[i+1])
            else:
                #Get from the standard intersect function
                postlist=intersect(sdict,splitquery[i],splitquery[i+1])
            
            
            if mergepost is not None:
                mergepost=mergeposting(mergepost,postlist)
            else:
                mergepost=postlist
    else:
        pass
    return mergepost  

The following is a test for the retrieval system
It builds the sorted token database from the dataset. Then uses the retrieval function to serve the query.

In [116]:
# Test
token=tokenizer('/content/gdrive/'+path,docfilename)
stoken=sort_token(token)

In [117]:
# toggle between 0 , 1 and 2 for using the "dictionary as string" memory optimisation
# 1 for optimisation with dictionary as string
# 2 for optimisation with dictionary as string and blocked storage with k = 4
# 0 for using the standard method
optimise = 2
docindex=retrieval("Border control",stoken,optimise)
docindex

[1]

In [118]:
d=list()
for i in docindex:
  d.append(docfilename[i])
print("Text document found: ",d)

Text document found:  ['5642.txt']
